In [8]:
import pandas as pd

# 读取TSV文件
df = pd.read_csv('./data/yue/train.tsv', sep='\t', encoding='utf-8')

# 通过列名访问某一列
sentences = df['sentence']  # 假设列名为 '列名'

In [9]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import HumanMessage, SystemMessage
api_key = "cd493791757b49fdbc0eeba1367608f0.xGgQTGgq7e7jIORn"
model=ChatOpenAI(
temperature=0.95,
model="glm-4-flash",
openai_api_key=api_key,
openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)
# 回答粤语或者普通话即可。
messages = [
SystemMessage(content="请识别下列是粤语还是普通话。如果是粤语，翻译成普通话"),
HumanMessage(content="我哋今晚去边度食好？"),
]
response = model.invoke(messages)

# 仅输出翻译后的文本
translated_text = response.content  # 从响应中提取文本字段
print(translated_text)  # 打印翻译后的结果

这是粤语。

翻译成普通话是：“我们今晚去哪里吃饭好？”


In [10]:
# 这里不需要管，是assemblyai的使用
from langchain_community.document_loaders import AssemblyAIAudioTranscriptLoader

# 初始化加载器
loader = AssemblyAIAudioTranscriptLoader(
    file_path="./data/yue/clips/common_voice_yue_31172849.mp3",
    api_key="bbe5a4fed2904483b6d96ce840f7b452"
)

# 加载并转录音频
documents = loader.load()

# 处理转录后的文档
for doc in documents:
    print(doc.page_content)

Oto ho ho kei dingalem jung ying.


In [11]:
from pydub import AudioSegment
from aip import AipSpeech

# 百度云语音识别的 API Key 和 Secret Key
APP_ID = '6251315'
API_KEY = 'Yz15TnhVhSoiX3p1uUfOLYCX'
SECRET_KEY = '0HCL2dA6NIdK51Wz2jQVkqddwvnoAixb'

# 初始化AipSpeech对象
client = AipSpeech(APP_ID, API_KEY, SECRET_KEY)
# 设置 pydub 使用的 ffmpeg 路径
AudioSegment.ffmpeg = r"D:\FFmpeg\bin\ffmpeg.exe" 

def convert_mp3_to_wav(mp3_file_path, wav_file_path):
    """将 MP3 文件转换为 WAV 格式"""
    audio = AudioSegment.from_mp3(mp3_file_path)
    audio = audio.set_frame_rate(16000)  # 设置采样率为16000 Hz
    # audio.export(wav_file_path, format="wav")  # 使用 PCM 编码导出

    audio = audio.set_sample_width(2)  # 使用 2 字节采样（16-bit）
    audio = audio.set_channels(1)  # 设置为单声道（语音识别通常更适合单声道）
    audio = audio + 10  # 增加10dB

    audio.export(wav_file_path, format="wav", codec="pcm_s16le")  # 使用 PCM 编码导出

def speech_to_text(audio_file_path):
    """使用百度云语音识别将音频文件转为文本"""
    with open(audio_file_path, 'rb') as f:
        audio_data = f.read()
        
        result = client.asr(audio_data, 'wav', 16000, {
             'dev_pid': 1637,  # 语言参数设置为粤语
        })
        
        print(result)
        
        if result.get('err_no') == 0:
            return result['result'][0]
        else:
            return None

# 使用示例：首先将 MP3 文件转换为 WAV 文件
mp3_file_path = './data/yue/clips/common_voice_yue_31172849.mp3'
wav_file_path = 'converted_audio.wav'
convert_mp3_to_wav(mp3_file_path, wav_file_path)

# 然后将转换后的 WAV 文件传给百度云进行语音识别
transcription = speech_to_text(wav_file_path)
print(f"语音转文本结果：{transcription}")


{'corpus_no': '7457873985125832881', 'err_msg': 'success.', 'err_no': 0, 'result': ['我捉好好奇点解连中意我，'], 'sn': '614994189301736421600'}
语音转文本结果：我捉好好奇点解连中意我，


In [12]:
pi_key = "cd493791757b49fdbc0eeba1367608f0.xGgQTGgq7e7jIORn"
model=ChatOpenAI(
temperature=0.95,
model="glm-4-flash",
openai_api_key=api_key,
openai_api_base="https://open.bigmodel.cn/api/paas/v4/"
)

# 粤语到普通话翻译
messages = [
    SystemMessage(content="这是一句粤语，翻译成普通话，只输出翻译内容"),
    HumanMessage(content=transcription),  # 使用 transcription 变量替换原有文本
]

response = model.invoke(messages)

# 仅输出翻译后的文本
translated_text = response.content  # 从响应中提取文本字段
print(translated_text)

我为什么这么好奇为什么会喜欢我？
